In [7]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: gross_bookings_usd is already dropped, not right but okay
## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'position', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'position', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,target
0,73,31,219,219,121991,3,4.0,1,2.30,0.1467,...,26172,4,50,2,0,1,1,2559.420000,0,5
1,80,5,219,219,23410,2,3.0,1,2.20,0.0000,...,7274,2,24,1,0,1,0,391.870000,0,0
2,80,5,219,219,409,3,4.0,1,1.10,0.0000,...,7274,2,24,1,0,1,0,391.930000,0,0
3,112,5,219,219,30017,3,4.0,0,1.95,0.2247,...,18584,3,2,1,0,1,1,1638.690000,0,0
4,112,5,219,219,56497,2,2.5,0,2.48,0.0000,...,18584,3,2,1,0,1,1,1639.180000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12150,332589,5,219,219,63604,4,4.0,1,4.29,0.2206,...,13870,2,6,2,2,1,1,338.950000,0,5
12151,332589,5,219,219,2906,3,4.0,1,4.28,0.2114,...,13870,2,6,2,2,1,1,338.900000,0,0
12152,332610,19,158,219,24584,3,4.5,0,3.93,0.2839,...,3187,3,70,2,0,1,1,7875.390000,0,0
12153,332660,5,117,219,23500,4,4.5,1,4.22,0.0092,...,4562,1,7,2,0,1,0,1319.240583,1,0


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [5]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank") 

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100
)

[1]	valid_0's ndcg@1: 0.987486	valid_0's ndcg@2: 0.994399	valid_0's ndcg@3: 0.995156	valid_0's ndcg@4: 0.995205	valid_0's ndcg@5: 0.995205
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.987166	valid_0's ndcg@2: 0.994304	valid_0's ndcg@3: 0.995042	valid_0's ndcg@4: 0.995091	valid_0's ndcg@5: 0.995091
[3]	valid_0's ndcg@1: 0.98664	valid_0's ndcg@2: 0.994349	valid_0's ndcg@3: 0.994899	valid_0's ndcg@4: 0.994948	valid_0's ndcg@5: 0.994948
[4]	valid_0's ndcg@1: 0.98664	valid_0's ndcg@2: 0.99422	valid_0's ndcg@3: 0.994872	valid_0's ndcg@4: 0.994921	valid_0's ndcg@5: 0.994921
[5]	valid_0's ndcg@1: 0.986954	valid_0's ndcg@2: 0.994401	valid_0's ndcg@3: 0.995001	valid_0's ndcg@4: 0.99505	valid_0's ndcg@5: 0.99505
[6]	valid_0's ndcg@1: 0.987377	valid_0's ndcg@2: 0.994624	valid_0's ndcg@3: 0.995171	valid_0's ndcg@4: 0.99522	valid_0's ndcg@5: 0.99522
[7]	valid_0's ndcg@1: 0.987166	valid_0's ndcg@2: 0.994587	valid_0's ndcg@3: 0.995101	valid_0's ndcg@4: 0.99515	

LGBMRanker(metric='ndcg', objective='lambdarank')

In [9]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [10]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [21]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop('date_time', axis=1, inplace=True)

In [22]:
predictions = model.predict(test)

In [24]:
len(predictions)

4959183

In [25]:
submission = test[["srch_id", "prop_id"]]
submission['pred'] = predictions

<ipython-input-25-f49d9f7448f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['pred'] = predictions


In [26]:
del test

In [27]:
gc.collect()

260

In [45]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [46]:
submission.head(n=20)

,srch_id,prop_id,pred
23,1,99484,3.206651
26,1,128871,2.966503
25,1,128085,2.910900
20,1,90385,2.845331
22,1,95031,2.810122
28,1,139162,2.770829
24,1,123675,2.720407
18,1,82231,2.543289
12,1,61934,2.354568
21,1,94729,2.301610


In [48]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)